# BALTO Graphical User Interface (prototype)

This Jupyter notebook creates a GUI (graphical user interface) for BALTO (Brokered Alignment of Long-Tail Observations) project.  BALTO is funded by the NSF EarthCube program.  The GUI intends to provide an simplified and customizable method for users to access data sets of interest on servers that support the OpenDAP data access protocol.  The interactive GUI runs within the Jupyter notebook and uses the Python packages: <b>ipywidgets</b> and <b>ipyleaflet</b>.

## Set up a conda environment called "balto"

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to create a "balto" conda environment with the following commands.  

conda update -n base -c defaults conda <br>
conda create --name balto  <br>
conda activate balto  <br>
conda install -c conda-forge ipywidgets <br>
conda install -c conda-forge ipyleaflet <br>
conda install -c conda-forge pydap <br>

conda install -c conda-forge jupyterlab <br>
conda install -c conda-forge nb_conda_kernels  # (needed for conda envs) <br>

conda install -c conda-forge nodejs <br>
conda install -c conda-forge widgetsnbextension <br>
jupyter labextension install jupyter-leaflet <br>
jupyter labextension install @jupyter-widgets/jupyterlab-manager <br>

Change to directory with BALTO_GUI.ipynb. <br>
jupyter lab <br>

Finally, choose BALTO_GUI.ipynb in Jupyter Lab,
but make sure to choose the kernel:  Python [conda env:balto] <br>

References <br>
https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html <br>
https://jupyterlab.readthedocs.io/en/stable/user/extensions.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension<br>

## Import required packages

In [1]:
from ipyleaflet import Map
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, HTML
## from IPython.core.display import display
## from IPython.lib.display import display
# import pydap
from pydap.client import open_url
import requests   # for get_filenames()
import time      # for sleep

## Create the GUI components

This GUI is built up from <b>ipywidgets</b> (for the controls) and <b>ipyleaflet</b> (for the interactive map).

For more information on ipywidgets, see:  https://ipywidgets.readthedocs.io/en/latest/user_guide.html

For more information on ipyleaflet, see:
https://ipyleaflet.readthedocs.io/en/latest/


In [2]:
p0 = widgets.HTML(value=f"<p></p> <p></p>")   # padding
h0 = widgets.HTML(value=f"<b><font size=5>BALTO User Interface</font></b>")
# p0 = widgets.HTML(value="<p></p> <p></p>")   # padding
# h0 = widgets.HTML(value="<b><font size=5>BALTO User Interface</font></b>")
#---------------------------------------------
# h0 = widgets.Label('BALTO User Interface')

# Create an interactive map with ipyleaflets
# gui_width     = '640px'
gui_width     = '680px'
map_width     = '640px'   # (gui_width - 40px)
map_height    = '250px'
att_width     = '560px'
url_box_width = att_width
## url_box_width = '560px'
m = Map(center=(0.0, 0.0), zoom=1, 
        layout=Layout(width=map_width, height=map_height))

style0  = {'description_width': 'initial'}
style1  = {'description_width': '130px'}
style2  = {'description_width': '80px'}
style3  = {'description_width': '50px'}
# bbox_style = {'description_width': '130px'}
bbox_style = {'description_width': '100px'}
date_style = {'description_width': '70px'}

#####################################################
# Does "step=001" restrict accuracy of selection ??
#####################################################
bbox_width = '270px'
w1 = widgets.BoundedFloatText(
    value=-180, min=-180, max=180.0, step=0.01,
    # description='West longitude:',
    description='West edge lon:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w2 = widgets.BoundedFloatText(
    value=180, min=-180, max=180.0, step=0.01,
    # description='East longitude:',
    description='East edge lon:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w3 = widgets.BoundedFloatText(
    value=90, min=-90, max=90.0, step=0.01,
    # description='North latitude:',
    description='North edge lat:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w4 = widgets.BoundedFloatText(
    value=-90, min=-90, max=90.0, step=0.01,
    # description='South latitude:',
    description='South edge lat:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
# Date Range  (& Temporal Resolution ??)
date_width = '270px'
d1 = widgets.DatePicker( description='Start Date:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d2 = widgets.DatePicker( description='End Date:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d3 = widgets.Text( description='Start Time:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d4 = widgets.Text( description='End Time:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )

# Variable Options
n0 = widgets.HTML(value=f"<p> </p>")   # padding
n1 = widgets.Text(description='Variable name:',
                  value='sea surface temperature',
                  disabled=False, style=style0,
                  layout=Layout(width='550px') )             

#------------------------------
# Example GES DISC opendap URL
#------------------------------
# https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHE.05/2014/
# 091/3B-HHR-E.MS.MRG.3IMERG.20140401-S000000-E002959.0000.V05B.HDF5.nc
# ?HQprecipitation[1999:2200][919:1049],lon[1999:2200],lat[919:1049]

#----------------------------------
# Example OpenDAP URL for testing
#----------------------------------
# http://test.opendap.org/dap/data/nc/coads_climatology.nc

## value='http://test.opendap.org/dap/data/nc/coads_climatology.nc',
## value='https://gpm1.gesdisc.eosdis.nasa.gov/opendap/',
    
# OpenDAP Options
o1 = widgets.Text(description='OpenDAP Dir:',
                  value='http://test.opendap.org/dap/data/nc/',
                  disabled=False, style=style1,
                  layout=Layout(width=url_box_width))
b1 = widgets.Button(description="Go", layout=Layout(width='50px'))
o2 = widgets.Dropdown( description='Filename:',
                       options=[''], value='',
                       disabled=False, style=style1,
                       layout=Layout(width=att_width) )
## o3 = widgets.Select( description='Variable:',
o3 = widgets.Dropdown( description='Variable:',
                       options=[''], value='',
                       disabled=False, style=style1,
                       layout=Layout(width=att_width) )
o4 = widgets.Text(description='Units:', style=style1,
                  value='', layout=Layout(width=att_width) )
o5 = widgets.Text(description='Shape:', style=style1,
                  value='', layout=Layout(width=att_width) )
o6 = widgets.Text(description='Dimensions:', style=style1,
                  value='', layout=Layout(width=att_width) )
o7 = widgets.Text(description='Data type:', style=style1,
                  value='', layout=Layout(width=att_width) )

# Settings
s1 = widgets.Dropdown( description='OpenDAP package:',
                       options=['pydap', 'netcdf4'],
                       value='pydap',
                       disabled=False, style=style1)

# Output File Format
f1 = widgets.Dropdown( description='Download Format:',
                       options=['HDF', 'netCDF', 'netCDF4', 'ASCII'],
                       value='netCDF',
                       disabled=False, style=style0)

## layout=Layout(width=map_width, height=map_height))

# Buttons at the bottom
b3 = widgets.Button(description="Download")

# Can use this for output
status = widgets.Text(description=' Status:', style=style3,
                      layout=Layout(width='380px') )

log = widgets.Textarea( description='', value='',
              disabled=False, style=style0,
              layout=Layout(width='560px', height='160px'))

## Define some GUI utility functions

In [3]:
def get_bounds():
    return [m.west, m.south, m.east, m.north]

#==================================================================
def get_start_date():
    
    if (d1.value is not None):
        return str(d1.value)    # Need the str().
    else:
        return 'None'

#==================================================================
def get_end_date():

    if (d2.value is not None):
        return str(d2.value)   # Need the str().
    else:
        return 'None'

#==================================================================
def get_variable_name():
    return o3.value
    ## return n1.value

#==================================================================
def get_opendap_package():
    return s1.value

#==================================================================
def get_output_format():
    return f1.value

#==================================================================
def list_to_string( array ):

    s = ''
    for item in array:
        s = s + item + '\n'
    return s

#==================================================================   
def print_choices():

    msg = [
    'bounds = ' + str(get_bounds()),
    'opendap package = ' + get_opendap_package(),
    'start date = ' + get_start_date(),
    'end date = ' + get_end_date(),
    'variable = ' + get_variable_name() ]
    log.value = list_to_string( msg )

#==================================================================


## Define some GUI event handling functions

## Set up the GUI event handlers

In [4]:
def download_data( dum ):
    status.value = 'Download button clicked.'
    print_choices()
    
#==================================================================
def show_bounds( **kwargs ):
    
    # Called by m.on_interaction().
    # Don't need to process separate events?
    w1.value = m.west
    w2.value = m.east
    w3.value = m.north
    w4.value = m.south

#==================================================================
def show_bounds2( **kwargs ):

    # events: mouseup, mousedown, mousemove, mouseover,
    #         mouseout, click, dblclick, preclick
    event = kwargs.get('type')
    # print('event = ', event)
    if (event == 'mouseup') or (event == 'mousemove') or \
       (event == 'click') or (event == 'dblclick'):
        w1.value = m.west
        w2.value = m.east
        w3.value = m.north
        w4.value = m.south
        
    # status.value = event
    
    # with output2:
    #   print( event ) 

#==================================================================
def get_opendap_url():
    directory = o1.value
    if (directory[-1] != '/'):
        directory += '/'
    filename = o2.value
    return directory + filename

#==================================================================
def update_filename_list( dummy=None ):

    #----------------------------------------------------
    # Note: This is called when "Go" button is clicked.
    #----------------------------------------------------
    ## opendap_dir = 'http://test.opendap.org/dap/data/nc/'
    opendap_dir = o1.value
    filenames   = get_filenames( opendap_dir )
    if (len(filenames) == 0):
        print('Error:  No data files found.')
        return
    #-----------------------------------
    # Update filename list & selection
    #-----------------------------------
    o2.options = filenames
    o2.value   = filenames[0]
    #------------------------------------
    # Update info for this file/dataset
    # which also calls show_var_info().
    #---------------------------------------------
    # Does this get called automatically due to:
    # o2.observe() call below ?
    #---------------------------------------------    
    ## show_dataset_info()  ########
    ## show_var_info()

#==================================================================
def open_dataset():
    ## from pydap.client import open_url   # (at top)
    opendap_url = get_opendap_url()
    dataset = open_url( opendap_url )
    return dataset

#==================================================================
def show_dataset_info( dummy=None ):

    if (o2.value == ''):
        ## update_filename_list()   # (doesn't work)
        return
    
    #---------------------------------------------------
    # Note: Not sure why "dummy" arg is required here.
    #---------------------------------------------------
    # Note: When this is called, the following are set
    #       as global variables in the notebook.
    #---------------------------------------------------
    global dataset, short_names, long_names, units_names
    global short_name_map, units_map
  
    dataset     = open_dataset()
    short_names = get_all_var_shortnames( dataset )
    long_names  = get_all_var_longnames( short_names)
    units_names = get_all_var_units( short_names ) 
    # print('### short_names =', short_names)
    # print('### long_names  =', long_names)
    # print('### units_names =', units_names)

    short_name_map = dict(zip(long_names, short_names ))
    units_map  = dict(zip(long_names, units_names ))
    #-------------------------------------------
    # Update variable list and selected value.
    #-------------------------------------------    
    o3.options = long_names
    o3.value   = long_names[0]
    ## print('In show_dataset_info((), o3.value =', o3.value)
    ## time.sleeppppppp(0.5)  # [seconds]  # didn't help
    #------------------------------------
    # Show other info for this variable
    #------------------------------------
    ## show_var_info()
    
#==================================================================
def show_var_info( dummy=None ):

    if (o3.value == ''):
        return
    
    short_name = short_name_map[ o3.value ]
    # print('#### short_name =', short_name)
    #----------------------------------------------
    # Note: long_name is selected from Dropdown.
    # o3.value = get_var_longname( short_name )
    #----------------------------------------------
    # LATER: Do this and pass var to get_var_*()?
    #----------------------------------------------    
    # var = dataset[ short_name ]
    o4.value = get_var_units( short_name )
    o5.value = get_var_shape( short_name )
    o6.value = get_var_dimensions( short_name )
    o7.value = get_var_dtype( short_name )
    return

    #---------------------------------------------------
    # Note: Not sure why "dummy" arg is required here.
    #---------------------------------------------------
    try:
        short_name = short_name_map[ o3.value ]
        # print('#### short_name =', short_name)
        #----------------------------------------------
        # Note: long_name is selected from Dropdown.
        # o3.value = get_var_longname( short_name )
        #----------------------------------------------
        o4.value = get_var_units( short_name )
        o5.value = get_var_shape( short_name )
        o6.value = get_var_dimensions( short_name )
        o7.value = get_var_dtype( short_name )
    except:
        dum = 0

#==================================================================  
def get_all_var_shortnames( dataset ):
    return list( dataset.keys() )

#==================================================================  
def get_all_var_longnames( short_names ):
    # Assume short_names is available as global var.
    # short_names = get_all_var_shortnames()
    
    long_names  = list()
    for name in short_names:
        try:
            long_name = get_var_longname( name )
            long_names.append( long_name )
        except:
            # Use short name if there is no long_name.
            long_names.append( name )
            # print('No long name found for:', name)
    return long_names

#==================================================================  
def get_all_var_units( short_names ):
    # Assume short_names is available as global var.
    # short_names = get_all_var_shortnames()
    
    units_names = list()
    for name in short_names:
        try:
            units = get_var_units( name )
            units_names.append( units )
        except:
            units_names.append( 'unknown' )
            # print('No units name found for:', name)
    return units_names

#==================================================================
def get_var_dimensions( short_name ):
    var = dataset[ short_name ]
    if hasattr(var, 'dimensions'):
        return str(var.dimensions)
    else:
        return 'No dimensions found.'

#==================================================================
def get_var_longname( short_name ):
    var = dataset[ short_name ]
    if hasattr(var, 'long_name'):
        return var.long_name
    else:
        return short_name

#==================================================================
def get_var_units( short_name ):
    var = dataset[ short_name ]
    if hasattr(var, 'units'):
        return var.units
    else:
        return 'unknown'

#==================================================================
def get_var_shape( short_name ):
    var = dataset[ short_name ]
    return str(var.shape)

#==================================================================
def get_var_dtype( short_name ):
    var = dataset[ short_name ]
    return str(var.dtype)

#==================================================================
def get_var_attributes( short_name ):
    var = dataset[ short_name ]
    if hasattr(var, 'attributes'):
        return var.attributes
    else:
        return 'No attributes found.'

#==================================================================
def get_filenames( opendap_dir):
    r = requests.get( opendap_dir )
    lines = r.text.splitlines()
    # n_lines = len(lines)
    filenames = list()
    for line in lines:
        if ('"sameAs": "http://' in line) and ('www' not in line):
            line = line.replace('.html"', '')
            parts = line.split("/")
            filename = parts[-1]
            filenames.append( filename )
    return filenames

#==================================================================


In [5]:
m.on_interaction( show_bounds )
b1.on_click( update_filename_list )
b3.on_click( download_data )
o2.observe( show_dataset_info )
o3.observe( show_var_info, names='value' )   ### NEED names='value' !!!!!!
## o3.observe( show_var_info )  ## This alone works intermittently.

## Create the GUI from the GUI components

In [6]:
#===========================
# Set up the UI layout: V1
#===========================
# v1 = widgets.VBox([w1, w2, w3, w4])
# v2 = widgets.VBox([d1, d2, n0, n1 ])
# h1 = widgets.HBox([v1, v2])
# h2 = widgets.VBox([o1, o2]) 
# h3 = widgets.HBox([b1, status])
# ui = widgets.VBox([p0, h0, m, h1, h2, h3, p0, log])

#======================================
# Set up the UI layout: V2: Accordion
#======================================
e1a = widgets.VBox([w1, w2])
e1b = widgets.VBox([w3, w4])
e1  = widgets.HBox( [e1a, e1b])
v1  = widgets.VBox( [m, e1] )
v2a = widgets.VBox([d1, d2])
v2b = widgets.VBox([d3, d4])
v2  = widgets.HBox([v2a, v2b])
v3  = widgets.VBox([n1])
## bb  = widgets.HBox([b1,b2])
bb  = widgets.HBox([o1, b1])  # directory + button
v4  = widgets.VBox([bb,o2,o3,o4,o5,o6,o7])
## v4  = widgets.VBox([o1,o2,b1,o3])
h3  = widgets.HBox([f1, n0, b3])
## v5  = widgets.VBox([h3, status, log])
v5  = widgets.VBox([h3, log])
v6  = widgets.VBox([s1])
# selected_index=None causes all cells to be collapsed
# acc = widgets.Accordion( children=[v1, v2, v3, v4, v5],
## acc = widgets.Accordion( children=[v4, v1, v2, v3, v5, v6],
acc = widgets.Accordion( children=[v4, v1, v2, v5, v6],
                         selected_index=None,
                         layout=Layout(width=gui_width) )
acc.set_title(0,'Browse Data')
acc.set_title(1,'Spatial Extent')
acc.set_title(2,'Date Range')
acc.set_title(3,'Download Data')
acc.set_title(4,'Settings')
#-----------------------------
# acc.set_title(3,'Variable')
# acc.set_title(4,'Download')
# acc.set_title(5,'Settings')
#--------------------------------------
# acc.set_title(0,'Spatial Extent')
# acc.set_title(1,'Date Range')
# acc.set_title(2,'Variable')
# acc.set_title(3,'OpenDAP Server')
# acc.set_title(4,'Download')

## ui = widgets.VBox([p0,h0,m,acc])
ui = widgets.VBox([p0,h0,acc])

## Display the GUI

In [7]:
gui_output = widgets.Output()
display(ui, gui_output)


Output()

## Some information for testing

In [8]:
# Geographic bounding box for state of Colorado
# Colorado_xmin = -109.060253
# Colorado_xmax = -102.041524
# Colorado_ymin = 36.992426
# Colorado_ymax = 41.003444